# Lab Exercise 3: Constrained Least Squares Unmixing

## Objectives
- Implement sum-to-one constrained least squares using Lagrange multipliers
- Understand and implement projected gradient descent
- Apply simplex projection for full constraints
- Compare different constraint strategies

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../src')

from data_loader import HyperspectralDataLoader
from visualization import HSIVisualizer
from optimization import HyperspectralUnmixer
from metrics import UnmixingEvaluator

%matplotlib inline

## Setup: Load Data

Load the data and previous results for comparison.

In [ ]:
# Load data
loader = HyperspectralDataLoader("../data")
hsi_data, ground_truth = loader.load_pavia()
Y, _ = loader.vectorize_data()
S = np.load('../data/extracted_endmembers.npy')
endmember_names = np.load('../data/endmember_names.npy', allow_pickle=True)

# Load unconstrained results
A_unconstrained = np.load('../data/unconstrained_abundances.npy')

print(f"Data loaded: Y{Y.shape}, S{S.shape}")
print(f"Unconstrained abundances: {A_unconstrained.shape}")

## Task 1: Sum-to-One Constrained Least Squares

**Mathematical Background:**

The sum-to-one constrained problem is:
$$\min_{\mathbf{A}} \|\mathbf{S}\mathbf{A} - \mathbf{Y}\|_F^2 \quad \text{subject to} \quad \mathbf{1}^T\mathbf{A} = \mathbf{1}^T$$

Using Lagrange multipliers, the solution for pixel $p$ is:
$$\mathbf{a}_p^* = (\mathbf{S}^T\mathbf{S})^{-1}\mathbf{S}^T\mathbf{y}_p + (\mathbf{S}^T\mathbf{S})^{-1}\mathbf{1}\lambda_p^*$$

In [ ]:
def sum_to_one_constrained_manual(S, Y):
    """
    Manual implementation of sum-to-one constrained least squares.
    
    TODO: Implement using Lagrange multipliers
    """
    num_endmembers, num_pixels = S.shape[1], Y.shape[1]
    
    # Precompute matrices
    StS = S.T@S # YOUR CODE HERE
    StY = S.T@Y# YOUR CODE HERE
    ones = np.ones((num_endmembers,1)) # YOUR CODE HERE (vector of ones)
    
    try:
        StS_inv = np.linalg.inv(StS) # YOUR CODE HERE
    except np.linalg.LinAlgError:
        StS_inv = np.linalg.pinv(StS)
    
    # Unconstrained solution
    A_unconstrained = StS_inv @ StY # YOUR CODE HERE
    
    # Compute lambda for each pixel
    # lambda = (1 - 1^T A_unconstrained) / (1^T (S^T S)^(-1) 1)
    denominator = ones.T @ StS_inv @ ones # YOUR CODE HERE
    numerator = 1 - ones.T @ A_unconstrained # YOUR CODE HERE
    
    lambda_values = numerator / denominator
    
    # Final constrained solution
    A = A_unconstrained + StS_inv @ ones @ lambda_values  # YOUR CODE HERE
    
    return A

# Test implementation
A_sum_manual = sum_to_one_constrained_manual(S, Y)

# Compare with library
unmixer = HyperspectralUnmixer()
A_sum_library = unmixer.sum_to_one_constrained_ls(S, Y)

difference = np.max(np.abs(A_sum_manual - A_sum_library))
print(f"Implementation difference: {difference:.2e}")

# Check sum-to-one constraint
sums = np.sum(A_sum_library, axis=0)
print(f"Sum constraint violation: {np.max(np.abs(sums - 1)):.2e}")

## Task 2: Projected Gradient Descent Implementation

Implement projected gradient descent for different constraint types.

First, we will implement a Non negative Constrained Least Square.

In [ ]:
def projected_gradient_descent_manual(S, Y, max_iter=500, step_size=0.001, tolerance=1e-6):
    """
    Manual implementation of projected gradient descent.
    
    TODO: Implement the algorithm
    """
    num_endmembers, num_pixels = S.shape[1], Y.shape[1]

    ## Initialization 
    A = unmixer.unconstrained_least_squares(S, Y)
    A = np.maximum(A, 0) # YOUR CODE HERE

    # Precompute for efficiency
    StS = S.T@S# YOUR CODE HERE
    StY = S.T@Y# YOUR CODE HERE
    
    objective_values = []
    
    for iteration in range(max_iter):
        # Compute objective
        residual = S @ A - Y# YOUR CODE HERE
        objective = 0.5 * np.sum(residual**2) # YOUR CODE HERE
        objective_values.append(objective)
        
        # Compute gradient: grad_A = S^T (SA - Y)
        gradient = StS @ A - StY
        
        # Gradient descent step
        A_new = A - step_size * gradient
            
        # Apply projection based on constraint type
        A_new = np.maximum(A_new, 0) # YOUR CODE HERE
        
        # Check convergence
        if iteration > 0:
            rel_change = abs(objective_values[-1] - objective_values[-2]) / objective_values[-2]
            if rel_change < tolerance:
                print(f"Converged after {iteration+1} iterations")
                break
        
        A = A_new
    
    return A, objective_values

# Test implementation with non-negativity constraints
print("Testing non-negativity constrained unmixing...")

StS = S.T @ S
max_eig = np.max(np.linalg.eigvals(StS))
A_nn_manual, obj_nn = projected_gradient_descent_manual(S, Y, max_iter=100, step_size=.99/max_eig)

print(f"Final objective: {obj_nn[-1]:.6f}")
print(f"Minimum abundance: {np.min(A_nn_manual):.6f}")
print(f"Convergence in {len(obj_nn)} iterations")

plt.figure(figsize=(10,8))
plt.semilogy(obj_nn)
plt.xlabel("iterations")
plt.ylabel("objective")

### Evaluate the reconstruction using the function used in last Exercice.

In [ ]:

# TODO: Plot abundance maps
visualizer = HSIVisualizer()
evaluator = UnmixingEvaluator()
height, width = hsi_data.shape[:2]

# YOUR CODE HERE to plot abundance maps
visualizer.plot_abundance_maps(A_nn_manual, ground_truth, [height, width], endmember_names, cmap='viridis')
# YOUR CODE HERE

# TODO: Evaluate reconstruction
rgb_bands = loader.get_rgb_bands()
results = evaluator.evaluate_reconstruction(S,A_nn_manual, Y, [height, width])

print("Reconstruction Quality Metrics:")
print(f"  Mean SAM (degrees): {results['mean_sam_degrees']:.4f}")
print(f"  RMSE: {results['rmse']:.6f}")
if 'ssim_mean' in results:
    print(f"  Mean SSIM: {results['ssim_mean']:.4f}")
print(f"  SNR (dB): {results['snr_db']:.2f}")

# TODO: Reconstruct hyperspectral data
Y_reconstructed = S@A_nn_manual
hsi_reconstructed = Y_reconstructed.T.reshape(height, width, -1)

# TODO: Plot comparison
# YOUR CODE HERE
visualizer.plot_reconstruction_comparison(hsi_data, hsi_reconstructed, rgb_bands)

## Task 3: Implement Simplex Projection

The simplex projection is crucial for handling both non-negativity and sum-to-one constraints simultaneously.
We will first implement a simplex projection algorithm and then modify the projected gradient descent function to project the right constraint set.

### Implement the simplex projection

In [ ]:
def project_onto_simplex_manual(v):
    """
    Project vector v onto the unit simplex using Duchi et al. algorithm.
    
    TODO: Implement the simplex projection algorithm
    """
    n = len(v)
    
    # Step 1: Sort in descending order
    u = np.sort(v)[::-1] # YOUR CODE HERE
    
    # Step 2: Find rho
    cssv = np.cumsum(u) - 1.0 # YOUR CODE HERE (cumulative sum - 1)
    ind = np.arange(n) + 1 # YOUR CODE HERE (indices 1, 2, ..., n)
    cond = u - cssv / ind > 0 # YOUR CODE HERE (condition u - cssv/ind > 0)
    
    if np.any(cond):
        rho = np.where(cond)[0][-1] # YOUR CODE HERE
        theta = cssv[rho] / (rho + 1.0) # YOUR CODE HERE
    else:
        theta = 0.0
    
    # Step 3: Project
    w = np.maximum(v - theta, 0) # YOUR CODE HERE
    
    return w

# Test simplex projection
test_vector = np.array([0.5, -0.2, 0.8, 0.1])
projected = project_onto_simplex_manual(test_vector)

print(f"Original: {test_vector}")
print(f"Projected: {projected}")
print(f"Sum: {np.sum(projected):.6f}")
print(f"All non-negative: {np.all(projected >= 0)}")

# Compare with library implementation
projected_lib = unmixer.project_onto_simplex(test_vector)
print(f"Difference from library: {np.max(np.abs(projected - projected_lib)):.2e}")

### Modify the projected gradient descent algorithm to include multiple constraints.

In [ ]:
def projected_gradient_descent_manual(S, Y, constraint_type='simplex', 
                                    max_iter=500, step_size=0.001, tolerance=1e-6):
    """
    Manual implementation of projected gradient descent.
    
    TODO: Implement the algorithm
    """
    num_endmembers, num_pixels = S.shape[1], Y.shape[1]
    A = unmixer.unconstrained_least_squares(S, Y)

    # Initialize abundances based on constraint type
    if constraint_type == 'simplex':
        # Initialize on simplex
        for p in range(num_pixels):
                A[:, p] = project_onto_simplex_manual(A[:, p])# YOUR CODE HERE (You can project the unconstrained solution)
    elif constraint_type == 'non_negative':
        A = np.maximum(A, 0) # YOUR CODE HERE
    elif constraint_type == 'sum_to_one':
        A = A/np.sum(A, axis=0, keepdims=True)
    
    # Precompute for efficiency
    StS = S.T@S# YOUR CODE HERE
    StY = S.T@Y# YOUR CODE HERE
    
    objective_values = []
    
    for iteration in range(max_iter):
        # Compute objective
        residual = S @ A - Y# YOUR CODE HERE
        objective = 0.5 * np.sum(residual**2) # YOUR CODE HERE
        objective_values.append(objective)
        
        # Compute gradient: grad_A = S^T (SA - Y)
        gradient = StS @ A - StY
        
        # Gradient descent step
        A_new = A - step_size * gradient
            
        # Apply projection based on constraint type
        if constraint_type == 'non_negative':
            A_new = np.maximum(A_new, 0) # YOUR CODE HERE
        elif constraint_type == 'sum_to_one':
            A_new =  A_new / np.sum(A_new, axis=0, keepdims=True) # YOUR CODE HERE
        elif constraint_type == 'simplex':
            for p in range(num_pixels):
                A_new[:, p] = project_onto_simplex_manual(A_new[:, p]) # YOUR CODE HERE
        
        # Check convergence
        if iteration > 0:
            rel_change = abs(objective_values[-1] - objective_values[-2]) / objective_values[-2]
            if rel_change < tolerance:
                print(f"Converged after {iteration+1} iterations")
                break
        
        A = A_new
    
    return A, objective_values

# Test implementation with non-negativity constraints
print("Testing non-negativity constrained unmixing...")

StS = S.T @ S
max_eig = np.max(np.linalg.eigvals(StS))
A_nn_manual, obj_nn = projected_gradient_descent_manual(S, Y, 'non_negative', max_iter=100, step_size=.99/max_eig)

print(f"Final objective: {obj_nn[-1]:.6f}")
print(f"Minimum abundance: {np.min(A_nn_manual):.6f}")
print(f"Convergence in {len(obj_nn)} iterations")

plt.figure(figsize=(10,8))
plt.semilogy(obj_nn)
plt.xlabel("iterations")
plt.ylabel("objective")

## Task 4: Compare All Constraint Methods

Apply and compare all constraint strategies.

In [ ]:
# Apply all methods
print("Applying all constraint methods...")

methods = {
    'Unconstrained': A_unconstrained,
    'Sum-to-one': A_sum_library,
}

# Non-negativity constrained
A_nn, obj_nn = unmixer.projected_gradient_descent(S, Y, 'non_negative', max_iter=200)
methods['Non-negative'] = A_nn

# Fully constrained (simplex)
A_simplex, obj_simplex = unmixer.fully_constrained_least_squares(S, Y, max_iter=200)
methods['Simplex'] = A_simplex

# Evaluate all methods
evaluator = UnmixingEvaluator()
height, width = hsi_data.shape[:2]
rgb_bands = loader.get_rgb_bands()

comparison_results = {}
abundance_results = {}

for name, A in methods.items():
    # Reconstruction metrics
    results = evaluator.evaluate_reconstruction(S, A, Y, (height, width), rgb_bands)
    comparison_results[name] = results
    
    # Abundance constraint metrics
    abundance_stats = evaluator.evaluate_abundances(A)
    abundance_results[name] = abundance_stats
    
    print(f"\n{name} Results:")
    print(f"  SAM (degrees): {results['mean_sam_degrees']:.4f}")
    print(f"  RMSE: {results['rmse']:.6f}")
    print(f"  Fraction negative: {abundance_stats['fraction_negative']:.4f}")
    print(f"  Mean sum deviation: {abundance_stats['mean_sum_deviation']:.4f}")

# Compare methods table
evaluator.compare_methods(comparison_results)

## Task 5: Visualize Convergence

Plot the convergence behavior of iterative methods.

In [ ]:
# TODO: Plot convergence curves
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Non-negativity convergence
axes[0].semilogy(obj_nn, 'b-', linewidth=2, label='Non-negative')
axes[0].set_xlabel('Iteration')
axes[0].set_ylabel('Objective Value')
axes[0].set_title('Non-negativity Constrained Convergence')
axes[0].grid(True, alpha=0.3)
axes[0].legend()

# Simplex convergence
axes[1].semilogy(obj_simplex, 'r-', linewidth=2, label='Simplex')
axes[1].set_xlabel('Iteration')
axes[1].set_ylabel('Objective Value')
axes[1].set_title('Simplex Constrained Convergence')
axes[1].grid(True, alpha=0.3)
axes[1].legend()

plt.tight_layout()
plt.show()

# Convergence analysis
print(f"Non-negative convergence rate: {(obj_nn[0] - obj_nn[-1])/obj_nn[0]:.4f}")
print(f"Simplex convergence rate: {(obj_simplex[0] - obj_simplex[-1])/obj_simplex[0]:.4f}")

## Task 6: Visualize Abundance Maps

Compare abundance maps from different methods.

In [ ]:
# TODO: Plot abundance maps for comparison
visualizer = HSIVisualizer()

# Plot simplex-constrained abundances (best method)
print("Simplex-Constrained Abundance Maps:")
visualizer.plot_abundance_maps(A_simplex, (height, width), endmember_names)

# Compare abundance statistics
print("\nAbundance Statistics Comparison:")
visualizer.plot_abundance_statistics(A_simplex, endmember_names)

## Task 7: Analyze Constraint Satisfaction

Quantitatively analyze how well each method satisfies physical constraints.

In [ ]:
# TODO: Create constraint satisfaction analysis
constraint_analysis = {}

for name, A in methods.items():
    # Non-negativity
    negative_count = np.sum(A < 0)
    total_values = A.size
    negative_fraction = negative_count / total_values
    
    # Sum-to-one
    sums = np.sum(A, axis=0)
    sum_violations = np.sum(np.abs(sums - 1) > 0.01)  # 1% tolerance
    sum_violation_fraction = sum_violations / len(sums)
    
    # Simplex constraint (both together)
    in_simplex = (A >= 0).all(axis=0) & (np.abs(sums - 1) < 0.01)
    simplex_fraction = np.sum(in_simplex) / len(sums)
    
    constraint_analysis[name] = {
        'negative_fraction': negative_fraction,
        'sum_violation_fraction': sum_violation_fraction,
        'simplex_fraction': simplex_fraction
    }

# Display results
print("Constraint Satisfaction Analysis:")
print("-" * 60)
print(f"{'Method':<15} {'Negative %':<12} {'Sum Viol %':<12} {'In Simplex %':<12}")
print("-" * 60)

for name, stats in constraint_analysis.items():
    print(f"{name:<15} {stats['negative_fraction']*100:<11.2f} "
          f"{stats['sum_violation_fraction']*100:<11.2f} "
          f"{stats['simplex_fraction']*100:<11.2f}")

## Task 8: Step Size Analysis

Investigate the effect of step size on convergence.

In [ ]:
# TODO: Test different step sizes
step_sizes = [0.0001, 0.001, 0.01, 0.1]
step_results = {}

# Use a subset of data for faster testing
Y_subset = Y[:, :1000]  # First 1000 pixels

for step in step_sizes:
    print(f"Testing step size {step}...")
    
    try:
        A_test, obj_test = unmixer.projected_gradient_descent(
            S, Y_subset, 'simplex', max_iter=100, step_size=step, tolerance=1e-6)
        
        step_results[step] = {
            'final_objective': obj_test[-1],
            'iterations': len(obj_test),
            'convergence_rate': (obj_test[0] - obj_test[-1]) / obj_test[0]
        }
        
    except Exception as e:
        print(f"  Failed with step size {step}: {e}")
        step_results[step] = {'failed': True}

# Display results
print("\nStep Size Analysis:")
print("-" * 50)
for step, result in step_results.items():
    if 'failed' not in result:
        print(f"Step {step}: Final Obj={result['final_objective']:.2e}, "
              f"Iters={result['iterations']}, Rate={result['convergence_rate']:.4f}")
    else:
        print(f"Step {step}: Failed")

In [ ]:
# Save best results for next notebook
np.save('../data/simplex_abundances.npy', A_simplex)
np.save('../data/constraint_comparison.npy', comparison_results)
print("Constrained unmixing results saved!")